In [2]:
import pandas as pd
from sqlalchemy import create_engine

usuario = 'uaopiloto'
contraseña = 'ap'
host = 'localhost'  # Por ejemplo, 'localhost'
base_de_datos = 'UAOPiloto'

# Crea una cadena de conexión
cadena_conexion = f'mysql+mysqlconnector://{usuario}:{contraseña}@{host}/{base_de_datos}'
# Crea una instancia del motor SQLAlchemy
motor = create_engine(cadena_conexion)

In [3]:
periodos = pd.read_sql('SELECT distinct periodo FROM MATRICULAS ORDER BY periodo', con=motor)
print('periodos:', periodos)

# se observa que todos los períodos terminan 1 y 3, que son los que hay que 
# verificar si un estudiando estuvo en al menos 2 seguidos sin matricularse y sin graduarse.

periodos:     periodo
0    201801
1    201803
2    201901
3    201903
4    202001
5    202003
6    202101
7    202103
8    202201
9    202203
10   202301
11   202303


In [5]:
estudiantes = pd.read_sql('SELECT distinct id_estudiante FROM MATRICULAS ORDER BY id_estudiante', con=motor)
print('estudiantes:', estudiantes)

estudiantes:      id_estudiante
0          2136689
1          2146669
2          2146707
3          2146709
4          2146710
..             ...
237        2236079
238        2236229
239        2236496
240        2236497
241        2237170

[242 rows x 1 columns]


In [6]:
matriculas = pd.read_sql('SELECT periodo, id_estudiante FROM MATRICULAS', con=motor)
print('matriculas:', matriculas)

matriculas:       periodo  id_estudiante
0      201801        2175510
1      201801        2175510
2      201801        2175510
3      201801        2175510
4      201801        2175510
...       ...            ...
7291   201803        2185767
7292   201803        2185767
7293   201803        2185767
7294   201803        2185767
7295   201803        2185767

[7296 rows x 2 columns]


In [13]:
# armamos matriz estudiantes x periodo.   1 si matrículo, 0 si no.

for e in estudiantes:

    # obtenemos el primer periodo en que hizo matricula
    respuesta = pd.read_sql('SELECT min(periodo) periodo FROM MATRICULAS where id_estudiante = '+ e, con=motor)
    print(respuesta)

    primer_periodo=respuesta.iloc[0,0]

    for p in 

        


   periodo
0   201801
primer: 201801
